In [28]:
from utils import *
import numpy as np

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


## R() and dR() unit test

In [38]:
n_hat = np.array([1,0,0])
theta =  np.pi/6 #rad

rot_mat_simp = R_simp(n_hat, theta)
print(rot_mat_simp)
angs = np.array([theta,0 ,0])
rot_mat = R(angs)
print(rot_mat)

print(R2Euler(rot_mat))

d_rot_mat_simp = dR_simp(n_hat,theta)
# print(d_rot_mat_simp)

[[ 1.         0.         0.       ]
 [ 0.         0.8660254 -0.5      ]
 [ 0.         0.5        0.8660254]]
[[ 1.         0.         0.       ]
 [-0.         0.8660254  0.5      ]
 [ 0.        -0.5        0.8660254]]
[-0.52359878  0.         -0.        ]


## Inline pointcloud rendering using Vedo

In [12]:
from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget

#NOTE: 
#	PCAP is a network capture(?) format from Wireshark/ network capture tool
		# https://binarymillennium.blogspot.com/2008/07/velodyne-lidar-sample-data-getting-pcap.html

location = 'C:/Users/Derm/2021-03-10-16-43-50_Velodyne-VLP-16-Data_garminSignage.txt'
cloud = np.loadtxt(open(location, "rb"), delimiter=",")
#remove all rows with NaN elements
cloud = cloud[~np.isnan(cloud).any(axis=1)]
plt1 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)
settings.embedWindow(backend='ipyvtk', verbose = True) #was this
# settings.useParallelProjection = True #makes view orthographic
# plt1.show(c, "Volpe Data Test", at = 0)#.close() #WAS THIS

f =np.array([10,10,2]) #fidelity in x, y, z
lim = np.array([-100,100,-100,100,-10,10])
# lim = np.array([-50,50,-50,50,-10,10])

subdivide_scan(cloud,plt1, bounds = lim, fid = f, draw_grid = True) 
ViewInteractiveWidget(plt1.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)


## Inline volumetric rendering using ipyvolume

In [ ]:
#simple demo
import ipyvolume
ds = ipyvolume.datasets.aquariusA2.fetch()
short = ds.data[:,:,:]
ipyvolume.quickvolshow(short, lighting=True)

In [ ]:
ans = np.linspace(1,12,8)
ans[2] = 0
print(ans)

test = ans[ans < 10]
print(test)
np.shape(test)[0]